In [1]:
# Enforces PEP-8 style guideline
!pip install --upgrade pip
!pip install flake8 pycodestyle_magic

# Adds upper path to import common module
import sys
sys.path.append('../../')

/bin/bash: pip: command not found
/bin/bash: pip: command not found


In [2]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [ ]:
"""
Version 1: TIMEOUT
- used methods: DP

"""
import math
from functools import lru_cache

INF = float('inf')


# Wraps math.log10 with exception handling of 0
def log10(f):
    if f == 0:
        return -INF

    # Will emit ValueError if f is out of domain of logarithm (value > 0)
    return math.log10(f)


# Returns the original sentence Q from given R in :queries
# maximizing the probability P(Q|R)
def solution(words, first, after, recognize, queries):
    # Add dummy string for index 0
    words = ['## ERROR ##'] + words
    w_len = len(words)

    # Take a log after merging :first into :after to be :after[0, *] == :first
    # then pad zeros :after[*, 0] to be zero
    after = [
        [-INF] + [log10(f) for f in row]
        for row in [first] + after
    ]

    # Take a log and pad zeros to be
    # :recognize[0, *] == :recognize[*, 0] == 0.0
    recognize = [
        [-INF] * len(recognize)
    ] + [
        [-INF] + [log10(f) for f in row] for row in recognize
    ]

    # Answer for each queries
    result = []
    for query in queries:
        # Convert each words to index in :words
        query = [words.index(w) for w in query.split()]
        q_len = len(query)

        # Save choices made to reconstruct output
        choices = [[None] * (w_len + 2) for _ in range(q_len + 2)]

        # Get the probability maximizing P(Q|R) = P(R|Q) * P(Q) / P(R)
        # but for all Q, P(R) is constant so just maximize P(R|Q) * P(Q)
        @lru_cache(maxsize=(w_len + 2) * (q_len + 2))
        def solve(segment, previous):
            if segment == q_len:
                return 0.0  # log10(1)

            ret = -INF  # log10(0)
            for next in range(1, w_len):
                temp = after[previous][next] \
                     + recognize[next][query[segment]] \
                     + solve(segment + 1, next)
                if temp > ret:
                    choices[segment][previous] = next
                    ret = temp

            return ret

        # Reconstruct the Q from :choices built by :solve()
        def reconstruct(segment, previous):
            if segment == q_len:
                return ''

            next = choices[segment][previous]
            return words[next] + ' ' + reconstruct(segment + 1, next)

        _ = solve(0, 0)  # Unused
        result.append(reconstruct(0, 0).strip())

    return result


# I/O
m, q = list(map(int, input().split()))
words = input().split()
B = list(map(float, input().split()))
T = [list(map(float, input().split())) for _ in range(m)]
M = [list(map(float, input().split())) for _ in range(m)]
queries = list(map(lambda s: s[s.index(' ')+1:], [input() for _ in range(q)]))
result = solution(words, B, T, M, queries)
for s in result:
    print(s)

In [3]:
"""
Version 2:
- used methods: DP with optimization

References:
- Solution by yeonghoey: https://algospot.com/judge/submission/detail/350237  # noqa: E501
- Solution by ene5135: https://algospot.com/judge/submission/detail/627587

"""
import sys
import math

INF = float('inf')


# Fast input
try:
    import IPython
except ImportError as _:
    # On judge system
    rl = sys.stdin.readline
else:
    # On jupyter notebook
    rl = input


# Wraps math.log10 with exception handling of 0
def log10(f):
    if f == 0:
        return -INF

    # Will emit ValueError if f is out of domain of logarithm (value > 0)
    return math.log10(f)


# Returns the original sentence Q from given R in :queries
# maximizing the probability P(Q|R)
def solution(words, first, after, recognize, queries):
    m = len(words)

    # Preprocess parameters
    first = [log10(f) for f in first]
    after = [[log10(f) for f in row] for row in after]
    recognize = [[log10(f) for f in row] for row in recognize]
    queries = [[words.index(w) for w in query.split()] for query in queries]

    # Returns Q for given R
    def solve(R):
        q = len(R)

        # Initialize cache
        cache = [[first[i] + recognize[i][R[0]] for i in range(m)]] \
              + [[-INF] * m for _ in range(q - 1)]  # noqa: E127
        choices = [[-1] * m for _ in range(q)]

        for segment in range(1, q):
            for prev in range(m):
                # Fail-fast
                sub = cache[segment - 1][prev]
                if sub == -INF:
                    continue

                for next in range(m):
                    temp = after[prev][next] \
                         + recognize[next][R[segment]] \
                         + sub
                    if temp > cache[segment][next]:
                        cache[segment][next] = temp
                        choices[segment][next] = prev

        ret = []

        # Reconstruct Q in backward
        best = max(cache[-1])
        prev = cache[-1].index(best)
        for segment in reversed(range(q)):  # Better readability than range(q-1, -1, -1)  # noqa: E501
            ret.append(words[prev])
            prev = choices[segment][prev]

        return ' '.join(reversed(ret))

    return [solve(query) for query in queries]


# I/O
m, q = map(int, rl().split())
words = rl().split()
B = [float(f) for f in rl().split()]
T = [[float(f) for f in rl().split()] for _ in range(m)]
M = [[float(f) for f in rl().split()] for _ in range(m)]
queries = [s[s.index(' ')+1:] for s in [rl() for _ in range(q)]]
result = solution(words, B, T, M, queries)
for s in result:
    print(s)

In [4]:
"""
Testing

"""
import cProfile
import traceback
import helpers as hlpr


f = hlpr.clock(hlpr.timeout(seconds=10)(solution))
test_cases = [
    ((['I', 'am', 'a', 'boy', 'buy'],
      [1.0, 0.0, 0.0, 0.0, 0.0],
      [
          [0.1, 0.6, 0.1, 0.1, 0.1],
          [0.1, 0.1, 0.6, 0.1, 0.1],
          [0.1, 0.1, 0.1, 0.6, 0.1],
          [0.2, 0.2, 0.2, 0.2, 0.2],
          [0.2, 0.2, 0.2, 0.2, 0.2]
      ],
      [
          [0.8, 0.1, 0.0, 0.1, 0.0],
          [0.1, 0.7, 0.0, 0.2, 0.0],
          [0.0, 0.1, 0.8, 0.0, 0.1],
          [0.0, 0.0, 0.0, 0.5, 0.5],
          [0.0, 0.0, 0.0, 0.5, 0.5]
      ],
      ['I am a buy', 'I I a boy', 'I am am boy']),
     (['I am a boy', 'I am a boy', 'I am a boy'])),
]

passed, failed = [], []
for idx, tc in enumerate(test_cases, start=1):
    case, expected = tc
    result = None
    try:
        result = f(*case)
        assert result == expected, \
            'wants {} but got {}'.format(hlpr.truncate(expected, 20), hlpr.truncate(result, 20))  # noqa: E501
    except Exception as err:
        fmt = 'Case {} FAIL: {}'
        print(fmt.format(idx, str(err)))
        traceback.print_exc()
        failed.append(idx)
    else:
        fmt = 'Case {} PASS'
        print(fmt.format(idx))
        passed.append(idx)

    print()

print()
if len(passed) == len(test_cases):
    print('All test(s) have passed')
else:
    fmt = '{} of {} test(s) failed: {}'
    print(fmt.format(len(failed), len(test_cases), ', '.join(map(str, failed))))  # noqa: E501

[0.00016260] "solution(['I', 'am', 'a', 'boy', 'buy'], [1.0, 0.0, 0.0, 0.0, 0.0], [[0.1, 0.6, 0.1, 0.1, 0.1], [0.1, 0.1, 0.6, 0.1, 0.... → "['I am a boy', 'I am a boy', 'I am a boy']"
Case 1 PASS


All test(s) have passed


In [ ]:
!pip install matplotlib
"""
Performance analysis
- x-axis: n
- y-axis: O(f(n))

"""
import matplotlib as plt

_